# Travel Itinerary Planner with AI Agent

This notebook demonstrates an AI-powered travel itinerary planner that uses Google's comprehensive suite of APIs to gather real-time information and create personalized travel plans.

## Features
- **Real-time Data Integration**: Uses multiple Google APIs for complete travel information
- **Intelligent Tool Calling**: Demonstrates LLM tool usage with proper logging and responses
- **Structured Output**: Returns structured travel itineraries using Pydantic models
- **Multi-turn Conversations**: Supports follow-up queries to modify itineraries
- **Accurate Time Zones**: Gets real local time using Google Time Zone API

## Google APIs Used
- **Google Gemini API**: For LLM chat and intelligent tool calling
- **Google Geocoding API**: For location coordinate lookup
- **Google Routes API**: For distance and travel time calculations  
- **Google Weather API**: For comprehensive weather forecasts
- **Google Time Zone API**: For accurate local time information

## Key Benefits
- ✅ **Comprehensive Google Integration**: Uses 5 different Google APIs
- ✅ **Real-time Data**: Live weather, traffic, location, and time data
- ✅ **Intelligent Planning**: AI-driven itinerary optimization with real data
- ✅ **Accurate Timing**: Real local time zones for destinations
- ✅ **Professional Setup**: Separate API keys for different service groups

In [1]:
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import FunctionTool
from llama_index.core.bridge.pydantic import BaseModel, Field
from typing import List
from datetime import datetime, timezone as dt_timezone
import requests
import time

## Import Required Libraries

In [2]:
class ItineraryItem(BaseModel):
    time: str = Field(description="Scheduled time for the activity")
    activity: str = Field(description="Description of the activity")
    location: str = Field(description="Location of the activity")
    weather: str = Field(description="Weather condition at the time")
    travel_duration: str = Field(description="Travel duration to the location")

class TravelItinerary(BaseModel):
    destination: str = Field(description="Destination city")
    date: str = Field(description="Travel date")
    items: List[ItineraryItem] = Field(description="List of scheduled activities")

## Data Models

Define Pydantic models for structured output formatting.

In [3]:
def get_current_time(timezone: str, google_api_key: str) -> dict:
    """Get the current time for a given timezone using Google Time Zone API"""
    # Get coordinates for the timezone location first
    coords = get_location_coordinates(timezone, google_api_key)

    # Get current UTC timestamp
    current_utc_timestamp = int(time.time())
    
    # Get timezone information using Google Time Zone API
    url = "https://maps.googleapis.com/maps/api/timezone/json"
    params = {
        'location': f"{coords['latitude']},{coords['longitude']}",
        'timestamp': current_utc_timestamp,
        'key': google_api_key
    }

    response = requests.get(url, params=params)
    data = response.json()

    if data['status'] == 'OK':
        total_offset_seconds = data['rawOffset'] + data['dstOffset']
        
        # Convert to local time by adding the total offset
        local_timestamp = current_utc_timestamp + total_offset_seconds
        local_time = datetime.fromtimestamp(local_timestamp, tz=dt_timezone.utc)

        return {
            "time": local_time.strftime("%Y-%m-%d %H:%M:%S"),
            "timezone": data['timeZoneName'],
            "timezone_id": data['timeZoneId'],
            "location": timezone,
            "utc_offset_hours": total_offset_seconds / 3600,
            "raw_offset": data['rawOffset'],
            "dst_offset": data['dstOffset']
        }
    else:
        raise Exception(f"Time Zone API error: {data.get('errorMessage', data.get('status', 'Unknown error'))}")


def get_weather(city: str, google_api_key: str) -> dict:
    """Get current weather for a city using Google Weather API"""
    # First get coordinates for the city
    coords = get_location_coordinates(city, google_api_key)

    # Get weather data using coordinates
    url = "https://weather.googleapis.com/v1/forecast/days:lookup"
    params = {
        'key': google_api_key,
        'location.latitude': coords['latitude'],
        'location.longitude': coords['longitude']
    }

    response = requests.get(url, params=params)
    data = response.json()

    if response.status_code == 200 and 'forecastDays' in data and data['forecastDays']:
        forecast = data['forecastDays'][0]
        daytime = forecast['daytimeForecast']

        return {
            "city": city,
            "weather": daytime['weatherCondition']['description']['text'],
            "temp_max": forecast['maxTemperature']['degrees'],
            "temp_min": forecast['minTemperature']['degrees'],
            "humidity": daytime['relativeHumidity'],
            "wind_speed": daytime['wind']['speed']['value'],
            "condition_type": daytime['weatherCondition']['type']
        }
    else:
        raise Exception(f"Weather API error: {data}")


def get_location_coordinates(location_name: str, google_api_key: str) -> dict:
    """Get latitude and longitude for a location using Google Geocoding API"""
    url = f"https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        'address': location_name,
        'key': google_api_key
    }
    response = requests.get(url, params=params)
    data = response.json()

    if data['status'] == 'OK' and data['results']:
        location = data['results'][0]['geometry']['location']
        return {
            'latitude': location['lat'],
            'longitude': location['lng'],
            'formatted_address': data['results'][0]['formatted_address']
        }
    else:
        raise Exception(f"Could not geocode location: {location_name}. Status: {data.get('status')}")

def calculate_distance(start: str, end: str, google_api_key: str) -> dict:
    """Calculate driving distance between two locations using Google Routes API"""
    # Get coordinates for both locations
    start_coords = get_location_coordinates(start, google_api_key)
    end_coords = get_location_coordinates(end, google_api_key)

    # Prepare the request payload for Routes API
    payload = {
        "origin": {
            "location": {
                "latLng": {
                    "latitude": start_coords['latitude'],
                    "longitude": start_coords['longitude']
                }
            }
        },
        "destination": {
            "location": {
                "latLng": {
                    "latitude": end_coords['latitude'],
                    "longitude": end_coords['longitude']
                }
            }
        },
        "travelMode": "DRIVE",
        "routingPreference": "TRAFFIC_AWARE",
        "computeAlternativeRoutes": False,
        "routeModifiers": {
            "avoidTolls": False,
            "avoidHighways": False,
            "avoidFerries": False
        },
        "languageCode": "en-US",
        "units": "METRIC"
    }

    # Make the request to Routes API
    url = "https://routes.googleapis.com/directions/v2:computeRoutes"
    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': google_api_key,
        'X-Goog-FieldMask': 'routes.duration,routes.distanceMeters'
    }

    response = requests.post(url, json=payload, headers=headers)
    data = response.json()

    if response.status_code == 200 and 'routes' in data and data['routes']:
        route = data['routes'][0]
        distance_meters = route['distanceMeters']
        duration_seconds = int(route['duration'].rstrip('s'))

        # Convert to more readable format
        distance_km = round(distance_meters / 1000, 2)
        duration_minutes = round(duration_seconds / 60)

        return {
            "start": start,
            "end": end,
            "distance": f"{distance_km} km",
            "duration": f"{duration_minutes} minutes",
            "distance_meters": distance_meters,
            "duration_seconds": duration_seconds
        }
    else:
        raise Exception(f"Routes API error: {data.get('error', {}).get('message', 'Unknown error')}")

# Create Itinerary Messages
Constructs a list of `ChatMessage` objects for the conversation, appending a new user query to any existing messages. This supports multi-turn conversations by maintaining context for follow-up requests, relying on tool calls for data retrieval.

In [4]:
def create_itinerary_messages(query: str, previous_messages: list = None) -> list:
    messages = previous_messages or []
    messages.append(ChatMessage(role="user", content=query))
    return messages

## Tool Functions

These functions serve as tools for the AI agent to gather real-time information.

In [5]:
def run_travel_itinerary_planner(initial_query: str, follow_up_query: str, destination: str, gemini_api_key: str, google_api_key: str) -> dict:
    """
    Main function to run the travel itinerary planner with AI tool calling.

    Args:
        initial_query: Initial travel planning query
        follow_up_query: Follow-up query to modify the itinerary
        destination: Travel destination city
        gemini_api_key: Google Gemini API key for LLM
        google_api_key: Google API key for other services (Geocoding, Routes, Weather, TimeZone)

    Returns:
        Dictionary containing initial and follow-up responses
    """
    # Define wrapper functions with proper names
    def get_current_time_wrapper(timezone: str) -> dict:
        return get_current_time(timezone, google_api_key)

    def get_weather_wrapper(city: str) -> dict:
        return get_weather(city, google_api_key)

    def calculate_distance_wrapper(start: str, end: str) -> dict:
        return calculate_distance(start, end, google_api_key)

    # Initialize tools
    time_tool = FunctionTool.from_defaults(fn=get_current_time_wrapper)
    weather_tool = FunctionTool.from_defaults(fn=get_weather_wrapper)
    distance_tool = FunctionTool.from_defaults(fn=calculate_distance_wrapper)
    tools = [time_tool, weather_tool, distance_tool]

    # Create tools_by_name mapping
    tools_by_name = {t.metadata.name: t for t in tools}

    print("Available tools:")
    for tool in tools:
        print(f"  - {tool.metadata.name}: {tool.metadata.description}")
    print()

    # Initialize LLM and structured LLM
    llm = GoogleGenAI(model="gemini-2.0-flash", api_key=gemini_api_key)
    structured_llm = llm.as_structured_llm(TravelItinerary)

    print("Starting travel itinerary planning...")
    print(f"Destination: {destination}")
    print(f"Initial Query: {initial_query}")
    print("-" * 80)

    # Create a more explicit prompt to encourage tool usage
    enhanced_initial_query = f"""
    {initial_query}

    IMPORTANT: You must use the available tools to gather real-time information. Please:
    1. First call get_current_time_wrapper with timezone "{destination}" to get the current local time
    2. Call get_weather_wrapper with city "{destination}" to get current weather forecast
    3. Call calculate_distance_wrapper to get distances between major attractions in {destination}

    After gathering this real information using the tools, create a detailed itinerary incorporating the actual time, weather, and distance data.
    """

    # Initial query with tool calling
    messages = create_itinerary_messages(enhanced_initial_query)
    print("Making initial LLM call with tools...")

    # Get initial response with tool calling
    tool_response = llm.chat_with_tools(tools, chat_history=messages)
    tool_calls = llm.get_tool_calls_from_response(tool_response, error_on_no_tool_call=False)

    print(f"Found {len(tool_calls)} tool calls")

    # Process tool calls
    if tool_calls:
        messages.append(tool_response.message)
        for i, tool_call in enumerate(tool_calls):
            tool_name = tool_call.tool_name
            tool_kwargs = tool_call.tool_kwargs
            print(f"Tool Call {i+1}: {tool_name}")
            print(f"   Parameters: {tool_kwargs}")

            try:
                tool_output = tools_by_name[tool_name].call(**tool_kwargs)
                print(f"   Response: {tool_output}")

                messages.append(ChatMessage(
                    role="tool",
                    content=str(tool_output),
                    additional_kwargs={"tool_call_id": tool_call.tool_id}
                ))
            except Exception as e:
                print(f"   Error: {e}")
                messages.append(ChatMessage(
                    role="tool",
                    content=f"Error: {str(e)}",
                    additional_kwargs={"tool_call_id": tool_call.tool_id}
                ))

        # Get final response after tool calls
        print("Getting final response after tool calls...")
        tool_response = llm.chat_with_tools(tools, chat_history=messages)
    else:
        print("No tool calls were made. Proceeding with direct generation.")

    # Generate structured output
    print("Generating structured itinerary...")
    try:
        # Use the latest message content for structured generation
        latest_content = tool_response.message.content if hasattr(tool_response.message, 'content') else str(tool_response.message)
        final_initial_response = structured_llm.complete(latest_content).raw
        print("Initial itinerary generated successfully")
    except Exception as e:
        print(f"Error generating structured output: {e}")
        # Fallback to direct structured generation
        final_initial_response = structured_llm.complete(enhanced_initial_query).raw

    # Add AI response to messages for multi-turn
    messages.append(ChatMessage(role="assistant", content=str(final_initial_response)))

    print("-" * 80)
    print(f"Follow-up Query: {follow_up_query}")

    # Enhanced follow-up query
    enhanced_follow_up_query = f"""
    {follow_up_query}

    Please use the available tools if needed to get current time or weather information for the restaurant reservation.
    """

    # Follow-up query with tool calling
    messages = create_itinerary_messages(enhanced_follow_up_query, messages)
    print("Making follow-up LLM call with tools...")

    follow_up_response = llm.chat_with_tools(tools, chat_history=messages)
    tool_calls = llm.get_tool_calls_from_response(follow_up_response, error_on_no_tool_call=False)

    print(f"Found {len(tool_calls)} tool calls for follow-up")

    if tool_calls:
        messages.append(follow_up_response.message)
        for i, tool_call in enumerate(tool_calls):
            tool_name = tool_call.tool_name
            tool_kwargs = tool_call.tool_kwargs
            print(f"Follow-up Tool Call {i+1}: {tool_name}")
            print(f"   Parameters: {tool_kwargs}")

            try:
                tool_output = tools_by_name[tool_name].call(**tool_kwargs)
                print(f"   Response: {tool_output}")

                messages.append(ChatMessage(
                    role="tool",
                    content=str(tool_output),
                    additional_kwargs={"tool_call_id": tool_call.tool_id}
                ))
            except Exception as e:
                print(f"   Error: {e}")
                messages.append(ChatMessage(
                    role="tool",
                    content=f"Error: {str(e)}",
                    additional_kwargs={"tool_call_id": tool_call.tool_id}
                ))

        # Get final response after follow-up tool calls
        print("Getting final follow-up response after tool calls...")
        follow_up_response = llm.chat_with_tools(tools, chat_history=messages)
    else:
        print("No tool calls were made for follow-up. Proceeding with direct generation.")

    # Generate structured follow-up output
    print("Generating structured follow-up response...")
    try:
        latest_content = follow_up_response.message.content if hasattr(follow_up_response.message, 'content') else str(follow_up_response.message)
        final_follow_up_response = structured_llm.complete(latest_content).raw
        print("Follow-up response generated successfully")
    except Exception as e:
        print(f"Error generating follow-up structured output: {e}")
        # Fallback to direct structured generation
        final_follow_up_response = structured_llm.complete(follow_up_query).raw

    print("Travel itinerary planning completed!")
    print("-" * 80)

    return {
        "initial_response": final_initial_response.model_dump(),
        "follow_up_response": final_follow_up_response.model_dump()
    }

## Main Travel Itinerary Planner Function

This function orchestrates the entire travel planning process using AI tool calling.

In [ ]:
# Configuration - Separate API keys for different services
destination = "Paris"
gemini_api_key = "gemini_api_key"  # Gemini API key
google_api_key = "google_api_key"  # Google services API key (Geocoding, Routes, Weather, TimeZone)

# Define queries
initial_query = f"Generate a one-day travel itinerary for {destination} with key activities, timings, weather, and travel durations."
follow_up_query = "Add a dinner reservation at a popular restaurant to the itinerary with travel duration."

print("="*80)
print("STARTING TRAVEL ITINERARY PLANNER")
print("="*80)

# Run the travel itinerary planner
result = run_travel_itinerary_planner(
    initial_query=initial_query,
    follow_up_query=follow_up_query,
    destination=destination,
    gemini_api_key=gemini_api_key,
    google_api_key=google_api_key
)

print("\n" + "="*80)
print("FINAL TRAVEL ITINERARY RESULTS")
print("="*80)

# Display results in a more readable format
initial = result["initial_response"]
follow_up = result["follow_up_response"]

print(f"\n🏙️  DESTINATION: {initial['destination']}")
print(f"📅 DATE: {initial['date']}")
print(f"\n📋 INITIAL ITINERARY ({len(initial['items'])} activities):")
for i, item in enumerate(initial['items'], 1):
    print(f"  {i}. {item['time']} - {item['activity']}")
    print(f"     📍 {item['location']}")
    print(f"     🌤️  {item['weather']}")
    if item['travel_duration'] != 'N/A':
        print(f"     🚗 Travel: {item['travel_duration']}")
    print()

print(f"📋 FOLLOW-UP ITINERARY ({len(follow_up['items'])} activities):")
for i, item in enumerate(follow_up['items'], 1):
    print(f"  {i}. {item['time']} - {item['activity']}")
    print(f"     📍 {item['location']}")
    if item['travel_duration'] != 'N/A':
        print(f"     🚗 Travel: {item['travel_duration']}")
    print()

print("="*80)
print("TRAVEL ITINERARY PLANNER COMPLETED SUCCESSFULLY!")
print("="*80)

STARTING TRAVEL ITINERARY PLANNER
Available tools:
  - get_current_time_wrapper: get_current_time_wrapper(timezone: str) -> dict

  - get_weather_wrapper: get_weather_wrapper(city: str) -> dict

  - calculate_distance_wrapper: calculate_distance_wrapper(start: str, end: str) -> dict


Starting travel itinerary planning...
Destination: Paris
Initial Query: Generate a one-day travel itinerary for Paris with key activities, timings, weather, and travel durations.
--------------------------------------------------------------------------------
Making initial LLM call with tools...
Found 4 tool calls
Tool Call 1: get_current_time_wrapper
   Parameters: {'timezone': 'Paris'}
   Response: {'time': '2025-08-11 14:55:00', 'timezone': 'Central European Summer Time', 'timezone_id': 'Europe/Paris', 'location': 'Paris', 'utc_offset_hours': 2.0, 'raw_offset': 3600, 'dst_offset': 3600}
Tool Call 2: get_weather_wrapper
   Parameters: {'city': 'Paris'}
   Response: {'city': 'Paris', 'weather': 'Sunny',

## Setup Instructions

Before running this notebook, ensure you have:

1. **Two Google API Keys**:
   - **Gemini API Key**: For LLM chat and tool calling
     - Enable "Generative Language API" (Gemini) in Google Cloud Console
   - **Google Services API Key**: For location and travel services
     - Enable "Geocoding API", "Routes API", "Weather API", and "Time Zone API"

2. **Required Packages**:
   ```bash
   pip install llama-index-llms-google-genai llama_index requests
   ```

3. **Google Cloud Console Setup**:
   - Create a project in Google Cloud Console
   - Enable the required APIs listed above
   - Create API keys and restrict them to the appropriate services
   - Copy the API keys to replace the placeholders below

**API Configuration**:
- `gemini_api_key`: For Gemini LLM functionality
- `google_api_key`: For Geocoding, Routes, Weather, and Time Zone APIs

Replace both API keys in the configuration section with your actual Google API keys.